In [ ]:
# !pip install ib_insync

In [ ]:
import ib_insync as IBrk

# Connect

In [ ]:
IBrk.util.startLoop()
ib = IBrk.IB()

try:    ib.connect()
except: ib.connect(port=4002)

# Orders

In [ ]:
ib.openTrades()

# Positions

In [ ]:
ib.positions()

# Market Order

In [ ]:
ib.placeOrder(
    IBrk.Stock(symbol='MSFT', exchange='SMART', currency='USD'),
    IBrk.MarketOrder(action='BUY', totalQuantity=100)
)

# Limit Order

In [ ]:
ib.placeOrder(
    IBrk.Stock(symbol='MSFT', exchange='SMART', currency='USD'),
    IBrk.LimitOrder(action='BUY', totalQuantity=100, lmtPrice=123.45)
)

# Iron Condor

In [ ]:
TICKER, EXP = 'MSFT', '2025-01-01'

CB_Stk, CB_Prm = 120,  5.00
CS_Stk, CS_Prm = 110, 10.00
PS_Stk, PS_Prm =  90, 10.00
PB_Stk, PB_Prm =  80,  5.00

quant = 1.00
limit = round(- CB_Prm + CS_Prm + PS_Prm - PB_Prm, 2)

In [ ]:
cons = [
    IBrk.Option(TICKER, EXP.replace('-',''), strike=CB_Stk, right='C', exchange='SMART', currency='USD'), 
    IBrk.Option(TICKER, EXP.replace('-',''), strike=CS_Stk, right='C', exchange='SMART', currency='USD'), 
    IBrk.Option(TICKER, EXP.replace('-',''), strike=PS_Stk, right='P', exchange='SMART', currency='USD'), 
    IBrk.Option(TICKER, EXP.replace('-',''), strike=PB_Stk, right='P', exchange='SMART', currency='USD'), 
]

qualify = ib.qualifyContracts(*cons)
if (len(qualify) == 4):

    ib.placeOrder(IBrk.Contract(exchange='SMART', symbol=TICKER, currency='USD', secType='BAG', comboLegs=[
        IBrk.ComboLeg(cons[0].conId, ratio=1, action='BUY',  exchange='SMART'), 
        IBrk.ComboLeg(cons[1].conId, ratio=1, action='SELL', exchange='SMART'), 
        IBrk.ComboLeg(cons[2].conId, ratio=1, action='SELL', exchange='SMART'), 
        IBrk.ComboLeg(cons[3].conId, ratio=1, action='BUY',  exchange='SMART'), 
    ]), IBrk.Order(action='BUY', totalQuantity=quant, orderType='LMT', lmtPrice=limit, transmit=True))

# Close Equity Positions

In [ ]:
for P in ib.positions():
    if (P.contract.secType=='STK'): 

        # ================ Cancel Orders ================ #
        for T in ib.openTrades(): 
            if (T.contract.secType=='STK') and (T.contract.symbol==P.contract.symbol):
                ib.cancelOrder(T.order)

        # ================ Close Positions ================ #
        con = P.contract
        con.exchange = 'SMART'

        if (0 < P.position):  ib.placeOrder(con, IBrk.MarketOrder('SELL', abs(P.position)))
        if (P.position < 0):  ib.placeOrder(con, IBrk.MarketOrder('BUY',  abs(P.position)))
    pass
pass